#### Get data from the UCI ML repository (watch out: it's *not always online*)

[The repository website](https://archive.ics.uci.edu/datasets)

In [1]:
# Get the dataset from the UCI ML repo
import ucimlrepo as uci_data
dataset_dict_raw = uci_data.fetch_ucirepo(id=492)

In [2]:
dataset_pd = dataset_dict_raw['data']['features']
target_pd = dataset_dict_raw['data']['targets']

In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [4]:
dataset_pd_test = dataset_pd.copy()

In [5]:
dataset_pd_test

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00
...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00


In [57]:
dt.isoweekday('monday')

TypeError: descriptor 'isoweekday' for 'datetime.date' objects doesn't apply to a 'str' object

In [6]:
from datetime import datetime as dt

dataset_pd_test['datetime_obj'] = pd.to_datetime(dataset_pd_test['date_time'])

In [7]:
dataset_pd_test['hour'] = dataset_pd_test['datetime_obj'].dt.time

In [8]:
dataset_pd_test

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,datetime_obj,hour
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,2012-10-02 09:00:00,09:00:00
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,2012-10-02 10:00:00,10:00:00
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,2012-10-02 11:00:00,11:00:00
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,2012-10-02 12:00:00,12:00:00
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,2012-10-02 13:00:00,13:00:00
...,...,...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,2018-09-30 19:00:00,19:00:00
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2018-09-30 20:00:00,20:00:00
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2018-09-30 21:00:00,21:00:00
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,2018-09-30 22:00:00,22:00:00


In [10]:
dt.isoweekday(dt.now())

2

In [17]:
dataset_pd_test['day_of_week'] = dataset_pd_test['datetime_obj'].dt.dayofweek

In [22]:
dataset_pd_test['day_of_month'] = dataset_pd_test['datetime_obj'].dt.day
dataset_pd_test['month'] = dataset_pd_test['datetime_obj'].dt.month

In [23]:
dataset_pd_test

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,datetime_obj,hour,day_of_week,day_of_month,month
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,2012-10-02 09:00:00,09:00:00,1,2,10
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,2012-10-02 10:00:00,10:00:00,1,2,10
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,2012-10-02 11:00:00,11:00:00,1,2,10
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,2012-10-02 12:00:00,12:00:00,1,2,10
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,2012-10-02 13:00:00,13:00:00,1,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,2018-09-30 19:00:00,19:00:00,6,30,9
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2018-09-30 20:00:00,20:00:00,6,30,9
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2018-09-30 21:00:00,21:00:00,6,30,9
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,2018-09-30 22:00:00,22:00:00,6,30,9


In [ ]:
dataset_pd_test['day_of_week'] = dt.isoweekday(dataset_pd_test['datetime_obj'] )

TypeError: descriptor 'isoweekday' for 'datetime.date' objects doesn't apply to a 'Series' object

In [31]:
dt.now()

datetime.datetime(2025, 4, 14, 23, 24, 34, 776586)

In [48]:
dt.time(dt.now()).strftime('%H:%M:%S')

'23:26:00'